In [1]:
import pandas as pd
import json
from pprint import pformat
import sqlite3
conn = sqlite3.connect('lotr.db')

In [39]:
with open('LTR.json') as f:
    data = json.load(f)

lotr_set = data['data']['cards']

In [40]:
drop_list = ['availability', 'edhrecRank', 'colorIdentity', 'foreignData', 'frameVersion', 'language', 'layout', 
    'printings', 'securityStamp', 'setCode', 'boosterTypes', 'hasAlternativeDeckLimit', 'edhrecSaltiness', 'flavorName', 
    'borderColor', 'isReprint', 'asciiName']

set_lists = {}
set_dicts = {}

for i, card in enumerate(lotr_set):
    # remove 'availability' key
    for key in drop_list:
        card.pop(key, None)
    
    for key in card:
        if not card[key]:
            card[key] = "None"
        if isinstance(card[key], list):
            if key not in set_lists.keys():
                set_lists[key] = []
            for item in card[key]:
                new_card = {}
                new_card[key] = card[key]
                new_card['name'] = card['name']
                new_card['number'] = card['number']
                new_card['uuid'] = card['uuid']
                set_lists[key].append(new_card)
        if isinstance(card[key], dict):
            if key not in set_dicts.keys():
                set_dicts[key] = []
            new_card = {}
            new_card = card[key]
            new_card['name'] = card['name']
            new_card['number'] = card['number']
            new_card['uuid'] = card['uuid']
            set_dicts[key].append(new_card)

print(set_lists.keys())
print(set_dicts.keys())


dict_keys(['finishes', 'frameEffects', 'keywords', 'promoTypes', 'supertypes', 'types', 'variations', 'colors', 'subtypes'])
dict_keys(['identifiers', 'legalities', 'purchaseUrls', 'leadershipSkills'])


In [41]:
all_keys = set().union(*(d.keys() for d in lotr_set))
print(len(all_keys), all_keys)

32 {'variations', 'keywords', 'promoTypes', 'subtypes', 'rarity', 'isPromo', 'manaCost', 'text', 'types', 'isAlternative', 'isFullArt', 'manaValue', 'colors', 'hasNonFoil', 'supertypes', 'identifiers', 'frameEffects', 'flavorText', 'name', 'power', 'toughness', 'convertedManaCost', 'uuid', 'legalities', 'type', 'leadershipSkills', 'finishes', 'hasFoil', 'isStarter', 'purchaseUrls', 'artist', 'number'}


In [42]:
lotr = {}
for key in all_keys:
    if key in set_lists or key in set_dicts:
        continue
    lotr[key] = [lotr_set[i].get(key, None) for i in range(len(lotr_set))]

In [33]:
lotr_df = pd.DataFrame(lotr)
lotr_df.head()

,rarity,isPromo,manaCost,text,isAlternative,isFullArt,manaValue,hasNonFoil,flavorText,name,power,toughness,convertedManaCost,uuid,type,hasFoil,isStarter,artist,number
0,mythic,None,{4},Indestructible\nWhen The One Ring enters the b...,None,True,4.0,None,None,The One Ring,None,None,4.0,9b3292e8-01ad-5489-8116-a27f8bdf1bc2,Legendary Artifact,True,True,Veli Nyström,0
1,common,None,{4}{W},This spell costs {2} less to cast if it target...,None,None,5.0,True,Wormtongue bared his teeth; and then with a hi...,Banish from Edoras,None,None,5.0,33413ff7-336f-5340-888f-cbca91e3caa1,Sorcery,True,None,Veli Nyström,1
2,rare,None,{1}{W}{W},Destroy all creatures with power 3 or greater....,None,None,3.0,True,The Tooks marched in with Pippin at their head...,The Battle of Bywater,None,None,3.0,40b249cb-e103-5c37-9dcb-1ec6a206f2f6,Sorcery,True,None,Tomas Duchek,2
3,uncommon,None,{3}{W},"When Bill the Pony enters the battlefield, cre...",None,None,4.0,True,None,Bill the Pony,1,4,4.0,d4f0c01f-9b7b-5233-9783-cabad66364a4,Legendary Creature — Horse,True,None,Christina Kraus,3
4,rare,None,{2}{W},"Vigilance\nWhenever an opponent casts a spell,...",None,None,3.0,True,"""Farewell, Aragorn! Go to Minas Tirith and sav...","Boromir, Warden of the Tower",3,3,3.0,15714ad7-584b-53f1-8ead-2f064dc5ed4c,Legendary Creature — Human Soldier,True,None,Yigit Koroglu,4


In [43]:
print(pformat(set_lists['keywords'][0]))

{'keywords': ['Indestructible'],
 'name': 'The One Ring',
 'number': '0',
 'uuid': '9b3292e8-01ad-5489-8116-a27f8bdf1bc2'}


In [44]:
print(pformat(set_lists))

{'colors': [{'colors': ['W'],
             'name': 'Banish from Edoras',
             'number': '1',
             'uuid': '33413ff7-336f-5340-888f-cbca91e3caa1'},
            {'colors': ['W'],
             'name': 'The Battle of Bywater',
             'number': '2',
             'uuid': '40b249cb-e103-5c37-9dcb-1ec6a206f2f6'},
            {'colors': ['W'],
             'name': 'Bill the Pony',
             'number': '3',
             'uuid': 'd4f0c01f-9b7b-5233-9783-cabad66364a4'},
            {'colors': ['W'],
             'name': 'Boromir, Warden of the Tower',
             'number': '4',
             'uuid': '15714ad7-584b-53f1-8ead-2f064dc5ed4c'},
            {'colors': ['W'],
             'name': 'Dawn of a New Age',
             'number': '5',
             'uuid': '8890df6e-d22e-5dc0-8730-03962a9f1bcb'},
            {'colors': ['W'],
             'name': 'Dúnedain Blade',
             'number': '6',
             'uuid': 'fe23cdbf-630c-56bc-8907-867bff6c7213'},
            {'color

In [47]:
for key in set_lists:
    print(key, len(set_lists[key]))

finishes 877
frameEffects 365
keywords 311
promoTypes 173
supertypes 241
types 464
variations 518
colors 501
subtypes 528


In [35]:
print(pformat(set_lists['finishes'][0]))

{'finishes': ['foil'],
 'name': 'The One Ring',
 'number': '0',
 'uuid': '9b3292e8-01ad-5489-8116-a27f8bdf1bc2'}


In [36]:
print(pformat(set_dicts))

{'identifiers': [{'mcmId': '701764',
                  'mcmMetaId': '417594',
                  'mtgjsonV4Id': 'c2532311-a3bd-5ded-ae7a-169dc969cf70',
                  'mtgoId': '113219',
                  'name': 'The One Ring',
                  'number': '0',
                  'scryfallId': '93de9042-cc62-4ade-8d8d-68fdbc84bfae',
                  'scryfallIllustrationId': 'a6795e84-6dd5-4795-a481-bc26b0b13ca4',
                  'scryfallOracleId': '3aa83ed2-f48b-4ce6-a614-2c54ddf50538',
                  'uuid': '9b3292e8-01ad-5489-8116-a27f8bdf1bc2'},
                 {'cardKingdomFoilId': '278981',
                  'cardKingdomId': '278674',
                  'mcmId': '717083',
                  'mcmMetaId': '420510',
                  'mtgArenaId': '84693',
                  'mtgjsonV4Id': 'f6e1c2a5-0cd4-5245-a152-3b0befee0828',
                  'mtgoId': '111912',
                  'name': 'Banish from Edoras',
                  'number': '1',
                  'scryfallId'

In [17]:

for k in set_dicts:
    print(k)
    print(set_dicts[k])
    #print(set_dicts[k][0].keys())
    #df = pd.DataFrame(set_dicts[k])
    #print(df.head())
    #lotr_df = lotr_df.drop(columns=set_dicts[k][0].keys())
    # df.to_sql(f'{k}', conn, if_exists='replace', index=False)


identifiers
[{'identifiers': {'mcmId': '701764', 'mcmMetaId': '417594', 'mtgjsonV4Id': 'c2532311-a3bd-5ded-ae7a-169dc969cf70', 'mtgoId': '113219', 'scryfallId': '93de9042-cc62-4ade-8d8d-68fdbc84bfae', 'scryfallIllustrationId': 'a6795e84-6dd5-4795-a481-bc26b0b13ca4', 'scryfallOracleId': '3aa83ed2-f48b-4ce6-a614-2c54ddf50538'}, 'name': 'The One Ring', 'number': '0', 'uuid': '9b3292e8-01ad-5489-8116-a27f8bdf1bc2'}, {'identifiers': {'cardKingdomFoilId': '278981', 'cardKingdomId': '278674', 'mcmId': '717083', 'mcmMetaId': '420510', 'mtgArenaId': '84693', 'mtgjsonV4Id': 'f6e1c2a5-0cd4-5245-a152-3b0befee0828', 'mtgoId': '111912', 'scryfallId': 'a4410076-e1fe-45f3-a0ca-a91ab0133ff4', 'scryfallIllustrationId': 'c8e9ebb5-308e-46c5-92d4-78cd02a08047', 'scryfallOracleId': 'e4508afd-7cc3-4176-8291-cfc2c155ac11', 'tcgplayerProductId': '499911'}, 'name': 'Banish from Edoras', 'number': '1', 'uuid': '33413ff7-336f-5340-888f-cbca91e3caa1'}, {'identifiers': {'cardKingdomFoilId': '278920', 'cardKingdomId

In [24]:
for k in set_lists:
    print(k)
    # print(pformat(set_lists[k][0]))
    # create dataframe and save it as a new table in lotr.db
    # df = pd.DataFrame(set_lists[k])
    # lotr_df = lotr_df.drop(columns=[k])
    # df.to_sql(f'{k}', conn, if_exists='replace', index=False)


finishes
frameEffects
keywords
promoTypes
supertypes
types
variations
colors
subtypes


In [ ]:
lotr_df.to_sql('cards', conn, if_exists='replace', index=False)

In [ ]:
print(pformat(set_lists['finishes']))

In [ ]:
# read data from faltantes.json
with open('faltantes.json') as _file:
    #faltantes = json.dumps(_file)
    faltantes = json.loads(_file.read())

In [ ]:
print(faltantes)
# for each key on json faltantes, create a new table on lotr.db


In [ ]:
for i in range(5):
    globals()[f'var{i}'] = f'Value for variable {i}'
    print(globals()[f'var{i}'])  # Prints 'Value for variable 0', 'Value for variable 1', etc.
